In [1]:
# -*- coding: utf-8 -*-
import logging
import datetime
from logging.handlers import TimedRotatingFileHandler
from logging.handlers import RotatingFileHandler
from sklearn.externals import joblib
from collections import OrderedDict
import sys
from dayu.hooks.oss_hook import OSSHook

from dayu.hooks.hive_server_hook import HiveServerHook
from dayu.hooks.hive_cli_hook import HiveCliHook

import pandas as pd 
import numpy as np
from fastFM import sgd
from scipy import sparse
from scipy.sparse import csc_matrix
import time
from sklearn.metrics import auc,accuracy_score,roc_curve,roc_auc_score
from sklearn.metrics import confusion_matrix,recall_score
from sklearn import  metrics
from sklearn.linear_model import LogisticRegression

def split_table_name(datain):
    new_cols = []
    for column in datain.columns:
        if(len(column.split('.'))<2):
            return datain
        tb_name, col_name = column.split('.')
        new_cols.append((column, col_name))
    datain = datain.rename(columns=dict(new_cols))
    return datain

def read_from_hive2(output_file_name,insql,dtype):
    filename = output_file_name
    filepath = curr_dir+filename
    hive = HiveServerHook("warehouse_hive")
    hive.to_csv(insql,filepath , delimiter=',',lineterminator='\n', output_header=True)
    outdata = pd.read_csv(filepath, header=0,dtype=dtype)
    # 去除列名中带有的表名
    outdata = split_table_name(outdata)
    return outdata
def date_time_sub(startTime,endTime,date_format):
    try:
        startTime= datetime.datetime.strptime(startTime,date_format)
        endTime= datetime.datetime.strptime(endTime,date_format)
        return (endTime - startTime).days
    except:
        pass
curr_date = str(datetime.datetime.now())[0:10]
logging.info('当前日期: %s'%(curr_date))
curr_dir = '/home/souche/qiongjiu/hgc/'
curr_date=str(datetime.datetime.now())[0:10]
hive_cli = HiveCliHook("warehouse_hive")

DAYU_HOME : /home/souche/projects/datacenter-etl-v2


/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


[2020-11-11 15:48:40,395] {driver:120} INFO - Generating grammar tables from /usr/lib/python3.5/lib2to3/Grammar.txt
[2020-11-11 15:48:40,424] {driver:120} INFO - Generating grammar tables from /usr/lib/python3.5/lib2to3/PatternGrammar.txt
[2020-11-11 15:48:42,961] {<ipython-input-1-72bb441ecf4d>:61} INFO - 当前日期: 2020-11-11


# 数据读取

In [2]:
sql = """
SELECT order_sn as car_id,
       brand_code,
       brand_name,
       series_code,
       series_name,
       model_code,
       model_name,
       cast(mileage/10000 AS DECIMAL(10,2)) AS mileage_std,
       color AS color_name,
       first_license_date as first_license_plate_date,
       province_name,
       city_name,
       cast(real_pay_amount/10000 AS DECIMAL(10,2)) AS price,
       date(pay_time) as pay_time,
       age AS license_month,
       residual,
       datediff("2020-12-31 00:00:00", pay_time) AS days,
       cast(year(first_license_date) AS INT) AS license_year
FROM db_data.mid_car_dfc_sale_order_flag_b
WHERE flag = 0
and trade_type !='B2B' 
and pay_time is not null
and pay_time<="2020-11-02"
and province_name is not null
and city_name is not null
"""
# 说明 load_data_from_hive 是我们自己改的从大禹取数的函数，不是官方读取函数。请自行选择
dtype={'model_code':str}
data= read_from_hive2('mid_car_dfc_purchase_order',sql,dtype)


[2020-11-11 15:48:43,069] {hiveserver2:138} INFO - Using database default as default
[2020-11-11 15:48:43,262] {hive_server_hook:112} INFO - Running query: 
SELECT order_sn as car_id,
       brand_code,
       brand_name,
       series_code,
       series_name,
       model_code,
       model_name,
       cast(mileage/10000 AS DECIMAL(10,2)) AS mileage_std,
       color AS color_name,
       first_license_date as first_license_plate_date,
       province_name,
       city_name,
       cast(real_pay_amount/10000 AS DECIMAL(10,2)) AS price,
       date(pay_time) as pay_time,
       age AS license_month,
       residual,
       datediff("2020-12-31 00:00:00", pay_time) AS days,
       cast(year(first_license_date) AS INT) AS license_year
FROM db_data.mid_car_dfc_sale_order_flag_b
WHERE flag = 0
and trade_type !='B2B' 
and pay_time is not null
and pay_time<="2020-11-02"
and province_name is not null
and city_name is not null

[2020-11-11 15:48:44,381] {hive_server_hook:162} INFO - Written 

[2020-11-11 15:49:56,944] {hive_server_hook:162} INFO - Written 880000 rows so far.
[2020-11-11 15:49:57,735] {hive_server_hook:162} INFO - Written 890000 rows so far.
[2020-11-11 15:49:58,466] {hive_server_hook:162} INFO - Written 900000 rows so far.
[2020-11-11 15:49:59,268] {hive_server_hook:162} INFO - Written 910000 rows so far.
[2020-11-11 15:50:00,050] {hive_server_hook:162} INFO - Written 920000 rows so far.
[2020-11-11 15:50:00,870] {hive_server_hook:162} INFO - Written 930000 rows so far.
[2020-11-11 15:50:01,637] {hive_server_hook:162} INFO - Written 940000 rows so far.
[2020-11-11 15:50:02,429] {hive_server_hook:162} INFO - Written 950000 rows so far.
[2020-11-11 15:50:03,240] {hive_server_hook:162} INFO - Written 960000 rows so far.
[2020-11-11 15:50:04,037] {hive_server_hook:162} INFO - Written 970000 rows so far.
[2020-11-11 15:50:04,859] {hive_server_hook:162} INFO - Written 980000 rows so far.
[2020-11-11 15:50:05,587] {hive_server_hook:162} INFO - Written 990000 rows 

In [6]:
import copy
raw_data = copy.deepcopy(data)


In [11]:
# 建模数据涉及车型 
len(data.model_code.unique())

32505

In [12]:
data['mileage_std'] = data['mileage_std'].astype('float')
data['price'] = data['price'].astype('float')
data['license_month'] = data['license_month'].astype('int')
data['residual'] = data['residual'].astype('float')

data.head()

,car_id,brand_code,brand_name,series_code,series_name,model_code,model_name,mileage_std,color_name,first_license_plate_date,province_name,city_name,price,pay_time,license_month,residual,days,license_year
0,13643647,brand-49,丰田,series-1072,汉兰达,054186468,2013款 汉兰达 2.7L 两驱7座探索版,12.0,3,2012-09-01,四川,成都,11.25,2020-10-26,98,0.342153,66,2012
1,11455539,brand-49,丰田,series-1072,汉兰达,054186468,2013款 汉兰达 2.7L 两驱7座探索版,8.0,4,2014-05-01,山西,太原,19.60,2018-07-01,51,0.596107,914,2014
2,12306403,brand-49,丰田,series-1072,汉兰达,054186468,2013款 汉兰达 2.7L 两驱7座探索版,5.5,3,2015-01-01,宁夏,固原,19.00,2019-06-03,54,0.577859,577,2015
3,11434767,brand-49,丰田,series-1072,汉兰达,054186468,2013款 汉兰达 2.7L 两驱7座探索版,4.0,4,2014-05-01,内蒙古,包头,18.80,2018-06-24,50,0.571776,921,2014
4,12286362,brand-49,丰田,series-1072,汉兰达,054186468,2013款 汉兰达 2.7L 两驱7座探索版,9.0,3,2014-02-01,广东,湛江,18.30,2019-06-12,65,0.556569,568,2014


In [13]:
# 读取 001 处理好的数据
model = pd.read_csv(curr_dir+"2020-11-02版车型参数及独热编码.csv", header=0, low_memory=False)

In [14]:
model.columns

Index(['model_code', 'model_year', 'new_car_price', 'wheel_base', 'length',
       'height', 'width', 'max_torque', 'max_power', 'engine_volume_l',
       ...
       'series_level53', 'series_level54', 'series_level55', 'series_level56',
       'series_level57', 'series_level58', 'series_level59', 'series_level60',
       'series_level61', 'series_level62'],
      dtype='object', length=180)

In [15]:
# 读取可估价的车型
model = model[['model_code', 'model_year', 'new_car_price','year_1', 'year_2', 'year_3', 'year_4', 'year_5', 'year_6',
                     'year_7', 'year_8', 'year_9', 'year_10', 'year_11', 'year_12', 'year_13', 
                     'year_14', 'year_15', 'year_16','rate','rate_count']]
print("合并前数据数目为:%d, 车型数量为：%d" % (data.shape[0], len(data['model_code'].unique())))
data['model_code'] = data['model_code'].astype('str')
data = pd.merge(data, model, on='model_code', how='inner')
print("<---------------------------------------->")
print("合并后数据数目为:%d, 车型数量为：%d" % (data.shape[0], len(data['model_code'].unique())))

合并前数据数目为:1756087, 车型数量为：32505
<---------------------------------------->
合并后数据数目为:1751604, 车型数量为：31464


In [16]:
# 不能参加估计的车型查看 
raw_data[~raw_data.model_code.isin(list(set(data.model_code)))]['model_name'].value_counts()

2012款 五菱荣光小卡 1.2L 双排基本型                     129
2011款 瑞风 2.4L政采版 手动豪华型                      117
2014款 道奇Ram 长角号 5.7L 自动 美规版                 113
2017款 道奇Ram 长角号 5.7L V8 美规版                  81
2014款 道奇Ram 皮卡RAM1500 5.7L 自动                80
2011款 瑞风 2.0L政采版 手动豪华型                       76
2011款 瑞风 2.8T穿梭 柴油舒适版HFC4DA1-2B1             73
2012款 五菱荣光小卡 1.5L 双排                         61
2013款 金旅考斯特 XML6700                          51
2012款 瑞风 1.9T穿梭 柴油舒适型HFC4DB1-2C              50
2014款 坦途 5.7L TRD 美规版                        49
2011款 HIACE 2.7L自动标准版13座                     46
2014款 金杯海狮 2.0L舒适型V19                        44
2014款 金杯海狮 2.0L豪华型V19                        40
2007款 柯斯达 2.7L汽油豪华版 20座                      40
2011款 经典全顺 柴油 短轴 多功能 中顶 6座                   39
2011款 金杯海狮 2.0L第五代翔运舒适型4G20D4B               36
2012款 长安星光4500 1.3L标准型4G13S1                 36
2017款 上汽大通V80 2.5T5挡手动傲运通国V短轴中顶5/6/10座       35
2012款 东风小康C37 1.4L豪华型                        31
2009款 经典全顺 2.8T柴油标准型短轴中顶JX493ZLQ3       

In [17]:
model[model['model_year']<=2005].shape

(8430, 21)

# 数据筛选

In [18]:
# # 去掉一些不合理的数据 （meimei版）
data_m = data[(data['mileage_std']>=0.01) & (data['mileage_std']<=50)]
data_m = data_m[(data_m['price']>=0.3) & (data_m['price']<=300)]
data_m = data_m[(data_m['license_month']>=1) & (data_m['license_month']<=180)]
# data_m = data_m[(data_m['residual']>=0.05) & (data_m['residual']<1)]
data_m = data_m[(data_m['model_year']>=2006)]
data_m = data_m[data_m['license_year']>=2005]
data_m = data_m[(data_m['new_car_price']>=1) & (data_m['new_car_price']<=500)]
print("增加限制后，数据数量为:%d, 车型数量为:%d" % (data_m.shape[0], len(data_m['model_code'].unique())))

增加限制后，数据数量为:1702828, 车型数量为:29430


In [19]:
#需要计算 meimei 
data_m['per_mile'] = round(data_m['mileage_std']/(data_m['license_month']/12),2)
data_m['year_err'] = data_m['license_year'] - data_m['model_year']
data_m = data_m[(data_m['year_err']>=-2) & (data_m['year_err']<=6)]
print("增加限制后，数据数量为:%d, 车型数量为:%d" % (data_m.shape[0], len(data_m['model_code'].unique())))

增加限制后，数据数量为:1702803, 车型数量为:29427


In [20]:
# 读取编码完成的省份数据
province = pd.read_csv("province_new.csv", header=0, low_memory=False)

In [21]:
data2m= pd.merge(data_m, province, how='left', on='province_name')

# 特征计算

In [22]:
# 计算车辆当前保值率
def computer_with_license_month(tar):
    license_month = tar['license_month']
    if(license_month<=12):
        #tar['keep_value'] = tar['year_1']
        return tar['year_1']
    else:
        year = license_month//12
        #当前年保值率
        keep_max = tar["year_"+str(int(year))]
        #下一年的保值率
        keep_min = tar["year_"+str(int(year+1))]
        
        #相比于上一年，已经过了几个月
        mon = license_month-12*year
        tem = (keep_max-keep_min)/12
        
        #tar['keep_value'] = round(keep_max - tem*mon, 4)
        return round(keep_max - tem*mon, 4)

#data2m = data2m.apply(computer_with_license_month, axis=1)

In [23]:
data2m['keep_value'] = data2m[['license_month','year_1', 'year_2', 'year_3', 'year_4','year_5', 'year_6', 'year_7', 'year_8', 'year_9', 'year_10', 'year_11','year_12', 'year_13', 'year_14', 'year_15', 'year_16']].to_dict(orient='records')
print("====")

====


In [24]:
data2m['keep_value'] = data2m['keep_value'].map(lambda tar:computer_with_license_month(tar))


In [25]:
data3m = data2m.drop(['year_1', 'year_2', 'year_3', 'year_4', 
                    'year_5', 'year_6', 'year_7', 'year_8', 
                    'year_9', 'year_10', 'year_11', 'year_12', 
                    'year_13', 'year_14', 'year_15', 'year_16'], axis=1)
quality = pd.read_csv(curr_dir+"2020-11-02版车型参数及独热编码.csv", header=0, low_memory=False)
quality = quality[['model_code', 'quality_mile', 'quality_year']]
quality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65250 entries, 0 to 65249
Data columns (total 3 columns):
model_code      65250 non-null object
quality_mile    65250 non-null float64
quality_year    65250 non-null float64
dtypes: float64(2), object(1)
memory usage: 1.5+ MB


In [26]:
data3m = pd.merge(data3m, quality, how='left', on='model_code')

In [27]:
def get_quality(license_month,mileage_std,quality_mile,quality_year):
    year = round(license_month/12, 2)
    if mileage_std < quality_mile and year< quality_year:
        return 1
    else:
        return 0
       
data3m['quality'] = list(map(lambda license_month,mileage_std,quality_mile,quality_year:get_quality(license_month,mileage_std,quality_mile,quality_year),
          data3m['license_month'],data3m['mileage_std'],data3m['quality_mile'],data3m['quality_year']))
data3m = data3m.drop(['quality_mile', 'quality_year'], axis=1)

In [30]:
#计算一下log  替换成 hive 计算  代码更新
# math.log 效率太低了，换成 np.log 之前是直接在取数的时候换的，以下code 需要测试确认
data3m['mile_log']=round(np.log2(data3m['mileage_std']), 4)
data3m['license_month_log']=round(np.log2(data3m['license_month']), 4)
data3m['new_car_price_log']=round(np.log2(data3m['new_car_price']), 4)

# 特征筛选2

In [32]:
# 根据per_mile过滤一下数据
data3m = data3m[data3m['per_mile']<=10]
data3m.shape

(1701843, 60)

In [34]:
# 数据筛选  查看每个车型对应的建模数据
data_counts_mei = data3m.groupby('model_code')['car_id'].count().rename('count').reset_index()
data5m = pd.merge(data3m, data_counts_mei, how='left', on='model_code')
data5m = data5m.reset_index(drop=True)
data5m.to_csv(curr_dir + "2020-11-02版带count处理完成的零售数据m.csv", header=True, index=False)

# 划分数据集

In [36]:
# 之前测试了两种 数据，所以有两条记录。

#交易条数超过5条
datahm = data5m[data5m['count']>=5]
#交易条数少于5条
datalm = data5m[data5m['count']<5]

print("交易条数超过5条的数据量是:%d, 车型数量是:%d" % (datahm.shape[0], len(datahm['model_code'].unique())))
print("交易条数少于5条的数据量是:%d, 车型数量是:%d" % (datalm.shape[0], len(datalm['model_code'].unique())))

交易条数超过5条的数据量是:1682754, 车型数量是:19969
交易条数少于5条的数据量是:19089, 车型数量是:9442


In [37]:
"""
单个model_code, 交易记录数超过5条, 使用
StratifiedShuffleSplit来进行划分, 测试
训练比例2:8; 单个model_code交易记录数少
于5条, 手动划分
"""
#step1:首先划分数据量超过5条的
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
stratified = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)


In [38]:
# 切meimei 的数据
for sub_set1, sub_set2 in stratified.split(datahm, datahm['model_code']):
    train_1m = datahm.iloc[sub_set1]
    test_1m = datahm.iloc[sub_set2]
print(train_1m.shape)
print(test_1m.shape)

(1346203, 61)
(336551, 61)


In [41]:
datalm['car_id'] = datalm['car_id'].map(lambda x:str(x))

/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [42]:
#按对应比例划分训练集和测试集
# 根据上面的订单号 ，修改函数里面的划分 
from sklearn.utils import shuffle
def split_data(data):
    model_code_list = list(data['model_code'].unique())
    #用来保存抽取的训练数据
    data_train = data[data['car_id']=="11145073"]
    #用来保存抽取的测试数据
    data_test = data[data['car_id']=="11377087"]
    for i in range(len(model_code_list)):
        model_code = model_code_list[i]
        if model_code=="14809-n":
            continue
        data_tem = data[data['model_code']==model_code]
        if(data_tem.shape[0]==1):
            data_train = pd.concat([data_train, data_tem])
        elif(data_tem.shape[0]==2):
            data_tem_test = data_tem.iloc[0:1, :]
            data_tem_train = data_tem.iloc[1:2, :]
            data_test = pd.concat([data_test, data_tem_test])
            data_train = pd.concat([data_train, data_tem_train])
        #三条以上的数据才能划分训练集和测试集
        else:
            data_tem = shuffle(data_tem)
            n = int(round(data_tem.shape[0]*0.2))
            data_tem_test = data_tem.iloc[0:n, :]
            data_tem_train = data_tem.iloc[n:, :]
            
            data_test = pd.concat([data_test, data_tem_test])
            data_train = pd.concat([data_train, data_tem_train])
        if(i%1000==0):
            print("已完成%d个model_code数据集划分, 共%d个, 进度%.2f%%" % 
                  (i, len(model_code_list), i/len(model_code_list)*100))
    return data_train, data_test


In [43]:
train_2m, test_2m = split_data(datalm)

已完成0个model_code数据集划分, 共9442个, 进度0.00%
已完成1000个model_code数据集划分, 共9442个, 进度10.59%
已完成2000个model_code数据集划分, 共9442个, 进度21.18%
已完成3000个model_code数据集划分, 共9442个, 进度31.77%
已完成4000个model_code数据集划分, 共9442个, 进度42.36%
已完成5000个model_code数据集划分, 共9442个, 进度52.95%
已完成6000个model_code数据集划分, 共9442个, 进度63.55%
已完成7000个model_code数据集划分, 共9442个, 进度74.14%
已完成8000个model_code数据集划分, 共9442个, 进度84.73%
已完成9000个model_code数据集划分, 共9442个, 进度95.32%


In [44]:
print(train_2m.shape)
print(test_2m.shape)

(13725, 61)
(5364, 61)


In [45]:
data_trainm = pd.concat([train_1m, train_2m])
data_trainm = data_trainm.reset_index(drop=True)

data_testm = pd.concat([test_1m, test_2m])
data_testm = data_testm.reset_index(drop=True)


In [46]:
# 输出 处理好的 训练集和测试集 
data_trainm.to_csv(curr_dir+"2020-11-02版零售数据data_train2.csv", header=True, index=False)
data_testm.to_csv(curr_dir+"2020-11-02版零售数据data_test2.csv", header=True, index=False)
